In [1]:
# Load packages
import requests 
import pandas as pd 
import time
import re

from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
url = 'https://www.hellopeter.com/uber-eats?page=1'  
# urls = [base_url + str(page) for page in range(1, 5)]
# print(urls)

In [13]:
resp = requests.get(url)
content = BeautifulSoup(resp.content, 'lxml')
print(content)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/><title>Review Companies Online | Customer Service &amp; Company Ratings | hellopeter.com</title><meta content="https://www.hellopeter.com/static/img/logo/hp-fb-crawl-3.png" property="og:image"/><link href="/static/img/logo/favicon-180x180.png?1" rel="apple-touch-icon" sizes="180x180"/><link href="/static/img/logo/favicon-32x32.png?1" rel="icon" sizes="32x32" type="image/png"/><link href="/static/img/logo/favicon-16x16.png?1" rel="icon" sizes="16x16" type="image/png"/><link color="#25a8e2" href="/static/img/safari-pinned-tab.svg" rel="mask-icon"/><link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.5.0/css/all.css" integrity="sha384-B4dIYHKNBt8Bc12p+WXckhzcICo0wtJAoU8YZTY5qE0Id1GSseTk6S+L3BlXeVIU" rel="stylesheet"/><link href="https://fonts.googleapis.com/css?family=Open+Sans:300,300i,400,400i,600,600i,70

In [34]:
def scrape_reviews(url):

  service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")
  driver = webdriver.Chrome(service=service)
  reviews = []

  driver.get(url)
    
  cards = driver.find_elements(By.CSS_SELECTOR, '.review-card')

  for card in cards:
  
    try:
      name = card.find_element(By.CSS_SELECTOR, '.has-text-weight-bold').text

      # Find date element using BeautifulSoup for more flexibility
      date_element = card.find_element(By.XPATH, ".//div[contains(text(), 'at')]")
      date_text = date_element.text

      # Extract date using BeautifulSoup and regex
      soup = BeautifulSoup(date_text, 'html.parser')
      date_match = re.search(r'(\d+ \w+ \d+ at \d+:\d+)', soup.get_text())
      date = date_match.group(1) if date_match else None
      
      rating = len(card.find_elements(By.CSS_SELECTOR, '.is-star-1'))
      
      title = card.find_element(By.CSS_SELECTOR, '.has-text-weight-bold.margin-top-24').text
      
      review = card.find_element(By.CSS_SELECTOR, '.margin-bottom-15').text
      
      review_data = {
        'review': review,
        'date': date, 
        'stars': rating,
        'name': name,
        'source': 'hellopeter',
        'title': title
      }

      reviews.append(review_data)
      
    except Exception as e:
      print(e)
      continue


  driver.quit()

  return pd.DataFrame(reviews)

In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re
import pandas as pd
import time

def scrape_reviews(url):

  service = Service(executable_path="C:\\Path\\to\\chromedriver.exe")

  driver = webdriver.Chrome(service=service)

  reviews = []

  driver.get(url)

  num_pages = 5

  for i in range(num_pages):

    # Wait for reviews to load
    time.sleep(5)

    cards = driver.find_elements(By.CSS_SELECTOR, '.review-card')

    for card in cards:

      try:
        name = card.find_element(By.CSS_SELECTOR, '.reviewer-name').text

        date_element = card.find_element(By.XPATH, ".//div[contains(text(), 'on')]")
        date_text = date_element.text

        soup = BeautifulSoup(date_text, 'html.parser')
        date = soup.select_one('div:nth-of-type(1)').text

        rating = len(card.find_elements(By.CSS_SELECTOR, '.fa-star.checked'))

        title = card.find_element(By.CSS_SELECTOR, '.review-title').text
        
        review = card.find_element(By.CSS_SELECTOR, '.review-content').text

        review_data = {
          'name': name,
          'date': date,
          'rating': rating,
          'title': title,
          'review': review
        }

        reviews.append(review_data)

      except Exception as e:
        print(e)
        continue

    # Click next button
    try:
      next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".next-page"))
      )
      next_button.click() 

    except:
      print("No more pages")
      break
      
    time.sleep(5)
      
  driver.quit()

  return pd.DataFrame(reviews)

# Example
url = 'https://www.hellopeter.com/uber-eats?page=1'  
scraped_reviews = scrape_reviews(url)
print(scraped_reviews)

NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [35]:
df_reviews = scrape_reviews(url)
df_reviews

,review,date,stars,name,source,title
0,You get no service with these people. You get ...,22 Jan 2024 at 11:23,1,Debbie R,hellopeter,UberEats the worst ever
1,I have tried to order from Uber eats 2 times n...,21 Jan 2024 at 21:56,1,Duduzile S,hellopeter,Horrible service
2,Uber eats is taking my money and I'm not recei...,21 Jan 2024 at 13:22,1,Abram M,hellopeter,Uber eats is a ****
3,Twice in one week I have placed an order on Ub...,20 Jan 2024 at 19:35,1,Mandi-Lee Maia,hellopeter,Ubereats
4,I'm disappointed about Uber eats I oder pizza ...,20 Jan 2024 at 13:24,1,Jan S,hellopeter,**** Uber eats ****mers
5,Ubereats is a dysfunctional company. Quick to ...,20 Jan 2024 at 07:07,1,pierre,hellopeter,Ubereats is useless. Rather use Mr delivery.
6,Im so disappointed with uber eats services and...,19 Jan 2024 at 22:13,1,Tinashemazarura80@gmail.com Mazarura,hellopeter,Complaint
7,I ordered a pizza and a 2lt Cream Soda from Pi...,19 Jan 2024 at 14:54,1,Darren,hellopeter,Uber*****s
8,I placed an order with Pedros Umhlanga via the...,19 Jan 2024 at 12:12,1,Rhoda P,hellopeter,Ubereats /Pedros umhlanga
9,"Do not ever use the Uber Eats for Grocery, unl...",19 Jan 2024 at 11:39,1,Vonanji K,hellopeter,Uber Eats Drivers ***** your groceries and Ube...


In [6]:
df_reviews.to_csv("ubereats_hellopeter.csv", index= False)
